내부 토폴로지가 복잡한 네트워크도 만들 수 있습니다. 비순환이라는 것이 중요합니다. 다시 말해 이 그래프는 원형을 띨 수 없습니다. 텐서 x가 자기 자신을 출력하는 층의 이력이 될 수 없습니다. 만들 수 있는 루프는 순환 층의 내부에 있는 것뿐입니다.

인셉션 모듈

삽성곱 신경망에서 인기있는 네트워크 구조입니다. 네트워크 안의 네트워크 구조에서 영감을 받아 만들었습니다. 이런 구성은 네트워크가 따로따로 공간 특성과 채널 방향의 특성을 학습하도록 돕습니다. 한꺼번에 학습하는 것보다 효과가 더 높습니다.

In [11]:
from keras import layers

branch_a = layers.Conv2D(128, 1, activation = 'relu', strides = 2)(x)
# 모든 가지는 동일한 스트라이드(2)를 사용합니다. 출력 크기를 동일하게 만들어 하나로 합치기 위해서입니다.

branch_b = layers.Conv2D(128, 1, activation = 'relu')(x)
branch_b = layers.Conv2D(128, 3, activation = 'relu', strides = 2)(branch_b)
# 이 가지에서는 두 번째 합성곱 층에서 스트라이드를 적용합니다.

branch_c = layers.AveragePooling2D(3, strides = 2)(x)
branch_c = layers.Conv2D(128, 3, activation = 'relu')(branch_c)
# 이 가지에서는 평균 풀링 층에서 스트라이드를 적용합니다.

branch_d = layers.Conv2D(128, 1, activation = 'relu')(x)
branch_d = layers.Conv2D(128, 3, activation = 'relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation = 'relu', strides = 2)(branch_d)
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis = -1)
# 모든 가지의 출력을 연결하여 모듈의 출력을 만듭니다.

NameError: name 'x' is not defined

엑셉션은 극단적인 인셉션(extreme inception)을 말합니다. 인셉션 V3와 거의 동일핫 개수의 모델 파라미터를 가지지만 실행 속도가 더 빠르고 대규모 데이터셋에서 정확도가 더 높습니다.

잔차 연결

대규모 딥러닝 모델에서 흔히 나타나는 두 가지 문제인 그래디언트 소실과 표현 병목을 해결했습니다. 일반적으로 10개 층 이상을 가진 모델에 잔차 연결을 추가하면 도움이 됩니다. 하위 층의 출력을 상위 층의 입력으로 사용합니다. 하위 층의 출력이 상위 층의 활성화 출력에 더해집니다. 따라서 두 출력의 크기가 동일해야 합니다. 

In [12]:
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(x)
# x에 어떤 변환을 적용합니다.
y = layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(y)
y = layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(y)

y = layers.add([y, x])
# 원본 x를 출력 특성에 더합니다.

ValueError: Layer conv2d_9 was called with an input that isn't a symbolic tensor. Received type: <class 'ellipsis'>. Full input: [Ellipsis]. All inputs to the layer should be tensors.

선형 변환을 사용하여 잔차 연결

In [13]:
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(x)
y = layers.Conv2D(128, 3, activation = 'relu', padding = 'same')(x)
y = layers.MaxPooling2D(2, strides = 2)(y)

residual = layers.Conv2D(128, 1, strides = 2, padding = 'same')(x)
# y와 크기를 맞추기 위해 1X1 합성곱을 사용하여 원본 텐서 x를 다운 샘플링합니다.

y = layers.add([y, residual])
# 다운샘플링된 x를 출력 특성에 더합니다.

ValueError: Layer conv2d_10 was called with an input that isn't a symbolic tensor. Received type: <class 'ellipsis'>. Full input: [Ellipsis]. All inputs to the layer should be tensors.

딥러닝의 표현 병목

어떤 층이 너무 작으면 이 활성화 출력에 얼마나 많은 정보를 채울 수 있느냐에 모델 성능이 좌우됩니다.

딥러닝의 그래디언트 소실 문제

피드백 신호가 깊이 쌓인 층을 통과하여 전파되면 신호가 아주 작아지거나 완전히 사라질 수도 있습니다.